# Profile Pro: A LinkedIn Profile Optimizer
## Final Project - Data Collection Lab (0940290)
### Lihi Kaspi (214676140), Harel Oved (326042389) & Lior Zaphir (326482213)

In [0]:
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import pandas as pd
from pyspark.ml.feature import CountVectorizer, Tokenizer, StringIndexer, VectorAssembler, Tokenizer, OneHotEncoder, Word2Vec, HashingTF, IndexToString
from pyspark.ml.linalg import SparseVector, Vectors
import numpy as np
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.window import Window
from datetime import datetime
import re
import shutil
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

spark = SparkSession.builder.getOrCreate()

## Relevant Data

In [0]:
profiles = spark.read.parquet('/dbfs/linkedin_people_train_data')
# new df with processed vector to go into the model
processed_data = spark.read.parquet("/Workspace/Users/lihi.kaspi@campus.technion.ac.il/processed_data.parquet")

In [0]:
processed_data = processed_data.withColumn(
    'label', 
    f.when(f.col('profile_score') < 5, 0
    ).when(f.col('profile_score') < 10, 1
    ).when(f.col('profile_score') < 15, 2
    ).when(f.col('profile_score') < 20, 3
    ).otherwise(4)
)

## Good Profiles Score Model

### Training the Model

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_accuracy = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy"
)

evaluator_f1 = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="f1"
)

In [0]:
train_df, test_df = processed_data.randomSplit([0.7, 0.3], seed=42)

In [0]:
from pyspark.ml.functions import vector_to_array

# Validate the training data
train_df = train_df.na.drop()
train_df = train_df.filter(f.size(vector_to_array(f.col('features'))) == 133)

# Validate the test data
test_df = test_df.na.drop()
test_df = test_df.filter(f.size(vector_to_array(f.col('features'))) == 133)

In [0]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

# Define the layers of the neural network
# Input layer = number of features, hidden layers = user-defined, output layer = number of classes
layers = [133, 64, 32, 5]

# Initialize MLP Classifier
mlp = MultilayerPerceptronClassifier(
    featuresCol="features",
    labelCol="label",
    maxIter=100,
    layers=layers,
    blockSize=128,
    seed=42
)

# Train the model
mlp_model = mlp.fit(train_df)

# Make predictions
mlp_predictions = mlp_model.transform(test_df)

# Evaluate
accuracy = evaluator_accuracy.evaluate(mlp_predictions)
f1_score = evaluator_f1.evaluate(mlp_predictions)

print(f"Accuracy: {accuracy}")
print(f"F1-Score: {f1_score}")

In [0]:
model_path = 'dbfs:/Workspace/Users/lihi.kaspi@campus.technion.ac.il/mlp_model'
if os.path.exists(model_path):
    shutil.rmtree(model_path)

mlp_model.write().overwrite().save(model_path)

In [0]:
import matplotlib.pyplot as plt

# Convert predictions to pandas DataFrame
sample = mlp_predictions.select('prediction').toPandas()

# Define the mapping of numeric predictions to category names
category_mapping = {0: 'bad', 1: 'below average', 2: 'average', 3: 'above average', 4: 'good'}

# Map the numeric predictions to category names
sample['category'] = sample['prediction'].map(category_mapping)

# Count the occurrences of each category
category_counts = sample['category'].value_counts().reindex(category_mapping.values(), fill_value=0)

# Plot the bar chart
plt.figure(figsize=(8, 6))
plt.bar(category_counts.index, category_counts.values, edgecolor='black', alpha=0.7, color='#0077B5')
plt.title('Histogram of Profile Scores Before Optimization')
plt.xlabel('Profile Score')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

## Profile Optimization

### 'about' Section Optimization

In [0]:
# take: about (if not null), position, job title, reccomendations 
# --> return: a sentence or two describing the person and job (in a new column called 'new_about')
# if all null: return message 'could not generate a short bio -- add more information to your profile' (put null in 'new_about' and add message in a new column called 'about_message')

good_profiles_df = ''







In [0]:
def generate_about(input_prompt, model, tokenizer):
    # Tokenize the input prompt
    inputs = tokenizer(input_prompt, return_tensors="pt", max_length=512, truncation=True)

    # Generate output text
    outputs = model.generate(inputs["input_ids"], max_length=150, num_beams=5, early_stopping=True)

    # Decode and return the generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
input_prompt = "City: New York, Education: Master's in Data Science from Columbia University, Name: Jane Doe, Position: Data Scientist"
about_section = generate_about(input_prompt, model, tokenizer)
print("Generated About Section:", about_section)


### Improvements and Suggetions

In [0]:
score_messages = {
    'excellent score': 'Your profile is excellent, keep it up!',
    'high score': 'Your profile is very strong, Check the suggestions to make it excellent',
    'medium high score': 'Your profile is good, Try to follow the suggestions to make it even better',
    'medium score': 'Your profile could use a few improvements, Try to follow the suggestions to make it even better',
    'medium low score': 'Your profile needs to improve, Try to follow the suggestion to make it better',
    'low score': 'Your profile is weak, Try to follow the suggestion to make it better',
}

In [0]:
score_messages = {
    100: 'Your profile is excellent, keep it up!', # good profiles with no suggestions
    4: 'Your profile is very strong, Check the suggestions to make it excellent',
    3: 'Your profile is good, Try to follow the suggestions to make it even better',
    2: 'Your profile could use a few improvements, Try to follow the suggestions to make it even better',
    1: 'Your profile needs to improve, Try to follow the suggestion to make it better',
    0: 'Your profile is weak, Try to follow the suggestion to make it better',
}

In [0]:
missing_field_messages = {
    'no_experience': 'Add previous/current comapnies you worked in', 
    'no_education': 'List your degrees and schools you graduated from',
    'no_about': 'Add a short bio about yourself, here is a suggestion: ', # for profiles with no about section at all
    'suggested_about': 'Try out this about section: ', # for bad profiles only
    'no_company': 'Add the company you currently work in',
    'no_languages': 'List all the languages you know and the level of knowledge',
    'no_position': 'Add the position you are currently in',
    'no_posts': 'Tell your friends about projects you currently work on',
    'no_recommendations': 'Ask a colleague to write a few words about you',
    'missing_experience': 'There is a gap in your resume, Don\'t forget to add all of the previous comapnies you worked in', # TODO: cahnge if we do exlicit time periods
    'low_followers': 'Ask your colleagues and friends to follow you on LinkedIn!'
    }

In [0]:
# find if there are gaps in the experience array (name new column: 'gap_in_experience')
# TODO: Binary or explicit time period? 

In [0]:
predicted_df = predicted_df.withColumn('suggestions', f.array())

predicted_df = predicted_df.withColumn(
  'suggestions',
  f.array(
    f.when(
      f.size(f.col('education')) == 0, 
      missing_field_messages.get('no_education')),
    f.when(
      f.size(f.col('current_company')) == 0, 
      missing_field_messages.get('no_company')),
    f.when(
      f.size(f.col('languages')) == 0, 
      missing_field_messages.get('no_languages')),
    f.when(
      f.size(f.col('posts')) == 0, 
      missing_field_messages.get('no_posts')),
    f.when(
      f.col('recommendations_count') == 0, 
      missing_field_messages.get('no_recommendations')),
    f.when(
      f.col('about').isNull() & f.col('new_about').isNotNull(), 
      missing_field_messages.get('no_about') + f.col('new_about')),
    f.when(
      f.col('about').isNotNull() & f.col('new_about').isNotNull() & f.col('score') < 90, 
      missing_field_messages.get('suggested_about') + f.col('new_about')),
    f.when(
      f.col('about_message').isNotNull(), 
      f.col('about_message')),
    f.when(
      f.col('position').isNull(),
      missing_field_messages.get('no_position')),
    f.when(
      f.col('followers') < 20,
      missing_field_messages.get('low_followers')),
    f.when(
      f.size(f.col('experience')) == 0, 
      missing_field_messages.get('no_experience')), 
    f.when(
      f.col('gap_in_experience').isNotNull(), # TODO: adapt to binary or time period
      missing_field_messages.get('missing_experience'))
  )
)

In [0]:
# placeholder name for the predictions: predicted_df (has all the previous columns + score predictions)

predicted_df = predicted_df.withColumn(
  'score_rank', 
  f.when(f.col('prediction') == 4 & f.size(f.col('suggestions')) == 0, 100
  ).otherwise(f.col('prediction'))
)

predicted_df = predicted_df.withColumn(
  'score_message',
  score_messages.get(f.col('score_rank'))
)

In [0]:
optemized_df = predicted_df.select('name', 'id', 'url', 'score_rank', 'score_message', 'suggestions')
display(optemized_df)

### Re-Evaluating the Profiles Post Optimization

In [0]:
# preprocess the profiles with the optimized 'about' sections and let the model predict the new scores

In [0]:
# need to see how listening to suggestions improve score - present the result by increasing number of sugggestions